# Training the Deep Neural Network Model for Food ordering smart assistant.

### Importing the libraries

In [3]:
import nltk
import numpy
import tflearn 
import tensorflow
import json
import pickle

from tensorflow.python.framework import ops
from nltk.stem import WordNetLemmatizer
lemmatizer = WordNetLemmatizer()


Instructions for updating:
non-resource variables are not supported in the long term
curses is not supported on this machine (please install/reinstall curses for an optimal experience)


### Importing the dataset

In [4]:
with open('data.json') as file:
    data = json.load(file)
    

### Function to save an object using file

In [5]:
def save(dObj, sFilename):
      '''Given an object and a file name, write the object to the file using pickle.'''

      f = open(sFilename, "wb")
      p = pickle.Pickler(f)
      p.dump(dObj)
      f.close()


### Function to Preprocess text 

In [6]:
def preprocess(text):
    ''' Accepts text as a parameter and performs the following preprocessing: 1. Tokenization
    2. Lemmatization 3. Convert words to lower case. Returns preprocessed tokens as output.
    Stop words and punctuations provide valuable context and hence are not preprocessed.'''

    tokens = []
    words = nltk.word_tokenize(text)

    for word in words:
        tokens.append(lemmatizer.lemmatize(word.lower()))

    return tokens
    

### Preprocess the data

In [11]:
import nltk
nltk.download('wordnet')

[nltk_data] Downloading package wordnet to
[nltk_data]     C:\Users\USER\AppData\Roaming\nltk_data...
[nltk_data]   Unzipping corpora\wordnet.zip.


True

In [12]:
tokens = set()
labels = set()

document_x = []
document_y = []

for intent in data['intents']:
    for question in intent['questions']:
        words = preprocess(question)
        tokens.update(words)
        document_x.append(words)
        document_y.append(intent["label"]) 
        labels.add(intent['label'])


### Using One-Hot Encoding to create bag of words

In [13]:
X_train = []
y_train = []

tokens = list(tokens)
labels = list(labels)

class_zeros = [0 for _ in range(len(labels))]

for i, document in enumerate(document_x):
    bag = []
    class_row = class_zeros[:]

    for token in tokens:
        if token in document:
            bag.append(1)
        else:
            bag.append(0)

    
    class_row[labels.index(document_y[i])] = 1
    X_train.append(bag)
    y_train.append(class_row)


### Training the DNN with 2 hidden layers with each having 8 fully connected neurons and a output layer with softmax activation function

In [14]:
X_train = numpy.array(X_train)
y_train = numpy.array(y_train)

ops.reset_default_graph()

net = tflearn.layers.core.input_data(shape=[None, len(X_train[0])])
net = tflearn.layers.core.fully_connected(net, 8)
net = tflearn.layers.core.fully_connected(net, 8)
net = tflearn.layers.core.fully_connected(net, len(y_train[0]), activation="softmax")
net = tflearn.layers.estimator.regression(net)

model = tflearn.DNN(net)
model.fit(X_train, y_train, n_epoch=1000, batch_size=8, show_metric=True)


Training Step: 15999  | total loss: 0.24692 | time: 0.042s
| Adam | epoch: 1000 | loss: 0.24692 - acc: 0.9830 -- iter: 120/127
Training Step: 16000  | total loss: 0.22346 | time: 0.047s
| Adam | epoch: 1000 | loss: 0.22346 - acc: 0.9847 -- iter: 127/127
--


### Saving the model and model information

In [15]:
model_information = {
    "tokens": tokens,
    "labels": labels,
    "X_train": X_train,
    "y_train": y_train
}

save(model_information, "model_information")
model.save("model.tflearn")

INFO:tensorflow:C:\Users\USER\Desktop\AI\model.tflearn is not in all_model_checkpoint_paths. Manually adding it.
